In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
import os.path
from PIL import Image

**Load dataset**

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',batch_size=64,image_size=(32,32))

Creating a data pipeline


In [ ]:
data_itr = data.as_numpy_iterator()

In [ ]:
batch = data_itr.next()
batch[0].shape

In [ ]:
batch[1]

In [ ]:
fig,ax = plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

**Preprocess Data**

Dividing all the pixels with 255 to get values between 0 and 1

In [ ]:
data = data.map(lambda x,y:(x/255,y))

In [ ]:
scaled_itr = data.as_numpy_iterator()
batch = scaled_itr.next()

In [ ]:
batch[0].shape

**Splitting data**

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.1)+1
test_size = int(len(data)*.1)

In [ ]:
train_size+val_size+test_size

In [ ]:
train = data.take(train_size)
test = data.skip(train_size).take(test_size)
val = data.skip(train_size+test_size).take(val_size)

**Deep learning model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.python.keras import regularizers

In [ ]:
model = Sequential()


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.L2(l2=0.01), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

**Train**

In [ ]:
hist = model.fit(train,
                 epochs = 50,
                validation_data = val)

**Plot performance**

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

pre = Precision()
acc = BinaryAccuracy()
re = Recall()
for batch in test.as_numpy_iterator():
    X,y = batch
    yhat = model.predict(X)
pre.update_state(y,yhat)
re.update_state(y,yhat)
acc.update_state(y,yhat)


In [ ]:
print(f"Preciison {pre.result().numpy()}, Recall {re.result().numpy()}, Accuracy {acc.result().numpy()}")

**Saving model**

In [ ]:
# from tensorflow.keras.models import load_model
# model.save(os.path.join('models','Imageclassifier.h5'))